<a href="https://colab.research.google.com/github/farshidehkordi/Homework2_AI/blob/main/TP2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import os


# List files in the current directory
print(os.listdir())


['.config', 'drive', 'sample_data']


In [ ]:
import os

# List files in the current directory
print(os.listdir())

# Check if the uploaded file exists
if 'CUB_200_2011.tar' in os.listdir():
    print("File exists!")
else:
    print("File not found.")


['.config', 'drive', 'sample_data']
File not found.


In [ ]:
import tarfile

# open file
file = tarfile.open('/content/drive/MyDrive/CUB_200_2011.tar')



# extract files
file.extractall('./CUB_dataset')

# close file
file.close()


In [ ]:
import os
import os
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
import torch.utils.data as td
import torchvision as tv
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
from shutil import copyfile




# Function to create a directory if it doesn't exist
def make_dir(file_path):
    if not os.path.exists(file_path):
        os.makedirs(file_path)

# Function to separate images into training and test sets
def separate_train_test(dataset_path, train_path, test_path):
    class_index = 1
    for classname in sorted(os.listdir(dataset_path)):
        if classname.startswith('.'):
            continue
        make_dir(os.path.join(train_path, classname))
        make_dir(os.path.join(test_path, classname))
        i = 0
        for file in sorted(os.listdir(os.path.join(dataset_path, classname))):
            if file.startswith('.'):
                continue
            file_path = os.path.join(dataset_path, classname, file)
            if i < 15:  # Number of images for the test set
                copyfile(file_path, os.path.join(test_path, classname, file))
            else:
                copyfile(file_path, os.path.join(train_path, classname, file))
            i += 1
        class_index += 1







In [ ]:
dataset_path = './CUB_dataset/CUB_200_2011/images'
train_path = './CUB_dataset/train'
test_path  = './CUB_dataset/test'

# Call the function to separate the dataset into training and test sets
separate_train_test(dataset_path, train_path, test_path)

ResNet18 using random initialization.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize images
])

# Load the training and test datasets
train_dataset = ImageFolder(root='./CUB_dataset/train', transform=transform)
test_dataset = ImageFolder(root='./CUB_dataset/test', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the ResNet18 model with random initialization
model =resnet18(weights=None)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total:.2f}%')


Epoch [1/10], Loss: 6.058868746324019
Epoch [2/10], Loss: 5.261587950966575


ResNet18 using pretrained modelon CUB_200 dataset, but freezing all the convolution parameters

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18 , ResNet18_Weights

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize images
])

# Load the training and test datasets
train_dataset = ImageFolder(root='./CUB_dataset/train', transform=transform)
test_dataset = ImageFolder(root='./CUB_dataset/test', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the ResNet18 model with pre-trained weights
model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)


# Freeze all the convolution parameters
for param in model.parameters():
    param.requires_grad = False

# Replace the fully connected layer with a new one (unfrozen)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)  # Only optimize the fully connected layer

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total:.2f}%')


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 38.6MB/s]


Epoch [1/10], Loss: 5.124582522999156
Epoch [2/10], Loss: 4.462770146456632
Epoch [3/10], Loss: 3.947295668341897
Epoch [4/10], Loss: 3.5296648259596393
Epoch [5/10], Loss: 3.192490868134932
Epoch [6/10], Loss: 2.9160115744850854
Epoch [7/10], Loss: 2.6903091968189585
Epoch [8/10], Loss: 2.5028984637693927
Epoch [9/10], Loss: 2.344278058572249
Epoch [10/10], Loss: 2.2109826317700474
Accuracy on test set: 50.40%


In [ ]:
pip install kaggle

In [ ]:
import os
from google.colab import files
import kaggle

files.upload()

!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
ls ~/.kaggle

kaggle.json


In [ ]:
!kaggle datasets list

ref                                                         title                                           size  lastUpdated          downloadCount  voteCount  usabilityRating  
----------------------------------------------------------  ---------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
sudarshan24byte/online-food-dataset                         Online Food Dataset                              3KB  2024-03-02 18:50:30          23426        465  0.9411765        
sahilnbajaj/loans-data                                      Loans Data                                     213KB  2024-04-07 15:08:37            858         23  1.0              
sahirmaharajj/air-pollution-dataset                         Air Pollution Dataset                          213KB  2024-04-07 13:14:48            943         29  1.0              
sanyamgoyal401/customer-purchases-behaviour-dataset         Customer Purchases Behaviour Dataset         

In [ ]:
!kaggle competitions download -c imagenet-object-localization-challenge

 53% 81.5G/155G [15:50<14:17, 92.1MB/s]
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/kaggle/cli.py", line 70, in main
    out = args.func(**command_args)
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 997, in competition_download_cli
    self.competition_download_files(competition, path, force,
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 965, in competition_download_files
    self.download_file(response, outfile, quiet, not force)
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 1877, in download_file
    out.write(data)
OSError: [Errno 28] No space left on device


In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision
from torchvision.datasets import ImageNet
from torch.utils.data import DataLoader
imagenet_data = torchvision.datasets.ImageNet('path/to/imagenet_root/')

RuntimeError: The archive ILSVRC2012_devkit_t12.tar.gz is not present in the root directory or is corrupted. You need to download it externally and place it in path/to/imagenet_root/.

Resnet18 using random initialization by default for ImageNet dataset

In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageNet
from torch.utils.data import DataLoader
import torchvision.models as models

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 pixels
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # Normalize images
])

# Load the ImageNet dataset using ImageFolder
imagenet_data = torchvision.datasets.ImageNet('path/to/imagenet_root/')
data_loader = torch.utils.data.DataLoader(imagenet_data,
                                          batch_size=4,
                                          shuffle=True,
                                          num_workers=args.nThreads)

# Load the pre-trained ResNet model
model = models.resnet18(pretrained=True)

# Freeze all layers except the fully connected layer
for param in model.parameters():
    param.requires_grad = False

# Adapt the fully connected layer to match the number of classes in your dataset
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, len(train_dataset.classes))  # Change the output size of the fully connected layer

# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model (code for training the model goes here)

# Evaluate the model (code for evaluating the model goes here)


FileNotFoundError: [Errno 2] No such file or directory: 'path/to/train_dataset'

ResNet18 using the pre-trained model, but only freezing the
parameters in "layer1" on CUB_200 dataset

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18 , ResNet18_Weights
# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # Normalize images
])

# Load the training and test datasets
train_dataset = ImageFolder(root='./CUB_dataset/train', transform=transform)
test_dataset = ImageFolder(root='./CUB_dataset/test', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load the pre-trained ResNet18 model
model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

# Freeze parameters in "layer1"
for name, param in model.named_parameters():
    if 'layer1' in name:
        param.requires_grad = False

# Adapt the fully connected layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total:.2f}%')


Epoch [1/10], Loss: 4.809893293380737
Epoch [2/10], Loss: 3.3419612164930865
Epoch [3/10], Loss: 2.425194742462852
Epoch [4/10], Loss: 1.8303259918906472
Epoch [5/10], Loss: 1.4222531648115677
Epoch [6/10], Loss: 1.1195163646611301
Epoch [7/10], Loss: 0.8801681715791876
Epoch [8/10], Loss: 0.6957529464634982
Epoch [9/10], Loss: 0.5466226833516901
Epoch [10/10], Loss: 0.4296421061320738
Accuracy on test set: 70.10%


ResNet18 using the pre-trained model on CUB_200 dataset, but letting all the parameters (including the convolution
layers) be adjusted by backprop.

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18, ResNet18_Weights

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # Normalize images
])


# Load the training and test datasets
train_dataset = ImageFolder(root='./CUB_dataset/train', transform=transform)
test_dataset = ImageFolder(root='./CUB_dataset/test', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Set the number of classes
num_classes = len(train_dataset.classes)

# Load the pre-trained ResNet18 model
model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

# Replace the fully connected layer with a new one
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total:.2f}%')


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 88.4MB/s]


Epoch [1/10], Loss: 4.820118252147328
Epoch [2/10], Loss: 3.344348253770308
Epoch [3/10], Loss: 2.4277283200350674
Epoch [4/10], Loss: 1.8262756954539905
Epoch [5/10], Loss: 1.3999622988700866
Epoch [6/10], Loss: 1.0926928474686362
Epoch [7/10], Loss: 0.8541336245970292
Epoch [8/10], Loss: 0.6707344388961792
Epoch [9/10], Loss: 0.5150841322812167
